# Importação de bibliotecas

In [3]:
# importando bibliotecas necessárias
from dotenv import load_dotenv
import os
from groq import Groq
import re

# carregando arquivo de variáveis de ambiente com chave da api
load_dotenv()
print("importação funcionou")

importação funcionou


# Classes necessárias para o agente

In [85]:
cliente = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = cliente.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explique em 10 palavras o que é IMC",
        }
    ],
    model="llama-3.3-70b-versatile",
)

print(chat_completion.choices[0].message.content)


# modelo de agente desenvolvido por https://www.youtube.com/watch?v=hKVhRA9kfeM

class Agente:
    def __init__(self, cliente: Groq, prompt_inicial: str = "") -> None:
        self.cliente = cliente
        self.prompt_inicial = prompt_inicial
        self.mensagens: list = []
        if self.prompt_inicial:
            self.mensagens.append({"role": "system", "content": prompt_inicial})

    def __call__(self, mensagem=""):
        if mensagem:
            self.mensagens.append({"role": "user", "content": mensagem})
        resultado = self.execute()
        self.mensagens.append({"role": "assistant", "content": resultado})
        return resultado

    def execute(self):
        completion = cliente.chat.completions.create(
            model="llama3-70b-8192", messages=self.mensagens
        )
        return completion.choices[0].message.content


prompt_inicial = """
Você executará uma sequência de estados começando por <Pensamento>, <Ação>, <Pausa> e <Observação>.
No final da sequência a sua saída será uma <Resposta> e uma <Dica> de saúde.
Use o estado <Pensamento> para descrever seu raciocínio sobre uma pergunta.
Use <Ação> para executar um das tarefas disponíveis para você e então retorne <Pausa>.
A <Observação> será o resultado de uma execução das tarefas.

As ações disponivéis são:

calcular_imc:
exemplo, calcular_imc: (peso, altura)
Execute o cálculo e retorne o número do IMC - a linguagem utilizada será python com pontos flutuantes


classificacao_imc:
exemplo, classificacao_imc: 18,5
retorne a classificacao do IMC

Sessão de exemplo:
Pergunta: Qual o meu IMC?
Pensamento: qual seu peso e altura?
Ação: calcular_imc: 85, 1.70
Pausa

Uma nova interação será realizada assim:

Observação: 29.41

Pensamento: Preciso classificar o  IMC
Ação: classificacao_imc: 29.41
Pausa

Uma nova interação será realizada assim:

Observação: SOBREPESO

Se você tem a resposta, retorne ela como <Resposta>.

<Resposta>: O IMC do seu peso e altura está classificado como Sobrepeso, faça dieta e exercícios!

Agora é sua vez:
""".strip()


def calcular_imc(args) -> float:
    '''
    responsavel por calcular o IMC do seu peso e altura através da fórmula peso / altura * altura
    :param args:
    :return:
    '''
    args = args.split(",")
    peso = float(args[0])
    altura = float(args[1])
    retorno = peso / (altura * altura)
    return retorno

def classificacao_imc(imc: str) -> str:
    '''
    responsável por classificar o IMC do seu peso e altura com base na tabela de IMC
    :param imc:
    :return:
    '''
    imc = float(imc)
    if imc < 18.5:
        return "MAGREZA"
    if imc >= 18.5 and imc <= 24.9:
        return "NORMAL"
    if imc >= 25 and imc <= 29.9:
        return "SOBREPESO"
    if imc >= 30 and imc <= 39.9:
        return "OBESIDADE"
    if imc >= 40:
        return "OBESIDADE GRAVE"

# teste unitário das funções
print(classificacao_imc(calcular_imc("92, 1.99")))


Índice de Massa Corporal mede saúde e peso saudável.
NORMAL


# testando o agente criado

In [86]:
# testando o agente e o prompt do sistema criado
# instanciação de teste do agente

agente_imc_teste = Agente(cliente=cliente, prompt_inicial=prompt_inicial)

# testando o estado inicial do agente
resposta_do_agente = agente_imc_teste.execute()
print(resposta_do_agente)


resposta_do_agente = agente_imc_teste("Me chamo paulo, meu peso é 85 kilos e minha altura é 1,70 metros")
print(resposta_do_agente)

Vamos começar! 

Pergunta: Qual o seu peso e altura?

(Please respond with your weight and height, and I'll start the sequence)

 Waiting for input...
Olá Paulo!

<Pensamento>: Ah, agora que sei seu peso e altura, posso calcular o seu IMC.

<Ação>: calcular_imc: 85, 1.70
<Pausa>


# Criando a interação do usuário e a iteração do agente

In [90]:
ferramentas = ["calcular_imc", "classificacao_imc"]

def iteracao(repeticao = 10, prompt: str = "", verbose: bool = False):
    agente = Agente(cliente=cliente, prompt_inicial=prompt_inicial)
    proximo_prompt = prompt

    indice = 0
    while indice < repeticao:
        indice += 1
        resposta = agente(proximo_prompt)
        if verbose: print(resposta)

        if "<Pausa>" in resposta and "<Ação>" in resposta:
            action = re.findall(r"<Ação>: ([a-z_]+): (.+)", resposta, re.IGNORECASE)
            if verbose: print(action)
            ferramenta = action[0][0]
            if verbose: print(ferramenta)
            parametros = action[0][1]
            if verbose: print(parametros)


            if ferramenta in ferramentas:
                resultado_ferramenta = eval(f"{ferramenta}('{parametros}')")
                proximo_prompt = f"<Observação>: {resultado_ferramenta}"
            else:
                proximo_prompt = "<Observação>: Ferramenta não existente"

            if verbose: print(proximo_prompt)
            continue

        if "<Resposta>" in resposta:
            print(resposta)
            break

iteracao(prompt="Me chamo Paulo, meu peso é 70.5 kilos e minha altura é 1,70 metros. Qual meu IMC?")

<Resposta>: O IMC do seu peso e altura está classificado como NORMAL, você está dentro do peso ideal! Continue se cuidando e mantendo um estilo de vida saudável!

<Dica>: Alimente-se de forma equilibrada, pratique exercícios regularmente e beba muita água para manter a saúde em dia!
